# ใช้โมเดล pre-trained Yolo เพื่อตรวจจับวัตถุผ่านกล้องเว็บแคม

Here are the direct download links for the files:

* Weights: https://pjreddie.com/media/files/yolov3.weights
* Configuration file: https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg
* COCO class labels: https://github.com/pjreddie/darknet/blob/master/data/coco.names

In [1]:
import numpy as np
import cv2

# Load the pre-trained YOLO model
yolo = cv2.dnn.readNet('yolov3.cfg', 'yolov3.weights')

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# Load the classes
classes = []
with open('yolov3_coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

: 

In [ ]:
# Set the input size
input_size = (416, 416)

# Open the video stream
cap = cv2.VideoCapture(0)

# Loop through the frames
while True:
    # Read the frame
    ret, frame = cap.read()
    if not ret:
        break

    # Create a blob from the frame and run it through the network
    blob = cv2.dnn.blobFromImage(frame, 1/255, input_size, (0, 0, 0), swapRB=True, crop=False)
    yolo.setInput(blob)
    outputs = yolo.forward(yolo.getUnconnectedOutLayersNames())

    # Loop through the outputs and draw the bounding boxes
    boxes = []
    confidences = []
    class_ids = []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                width = int(detection[2] * frame.shape[1])
                height = int(detection[3] * frame.shape[0])
                x = int(center_x - width / 2)
                y = int(center_y - height / 2)
                boxes.append([x, y, width, height])
                confidences.append(confidence)
                class_ids.append(class_id)
                
    indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.5)
    for i in indices:
        x, y, width, height = boxes[i]
        class_id = class_ids[i]            
        confidence = confidences[i]            
        cv2.rectangle(frame, (x, y), (x + width, y + height), (255, 255, 0), 1)
        cv2.putText(frame, f'{classes[class_id]} ({confidence:.4f})', (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display the output
    cv2.imshow('YOLO', frame)
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video stream and close the window
cap.release()
cv2.destroyAllWindows()

: 

: 